# Preparing the Data

In [7]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,:'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# print(readLines('data/names/Chinese.txt'))

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['data/names/Arabic.txt', 'data/names/Chinese.txt', 'data/names/Czech.txt', 'data/names/Dutch.txt', 'data/names/English.txt', 'data/names/French.txt', 'data/names/German.txt', 'data/names/Greek.txt', 'data/names/Irish.txt', 'data/names/Italian.txt', 'data/names/Japanese.txt', 'data/names/Korean.txt', 'data/names/Polish.txt', 'data/names/Portuguese.txt', 'data/names/Russian.txt', 'data/names/Scottish.txt', 'data/names/Spanish.txt', 'data/names/Vietnamese.txt']
Slusarski


In [10]:
print(category_lines['Chinese'][:5])

['Ang', 'AuYong', 'Bai', 'Ban', 'Bao']


## Turning Names into Tensors

In [15]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# ,
# or an array of one-hot letter vectors
def lineToTensor(line):
    """Turn a line into a <line_length x 1 x n_letters> Tensor"""
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))
print(lineToTensor('Jones'))


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0.

# Creating the Network

In [178]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, hidden):
        print('input:', input, input.size())
        print('hidden', hidden, hidden.size())
        combined = torch.cat((input, hidden), 1)
        print('combined', combined)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [65]:
# import torch.nn as nn

# class RNN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(RNN, self).__init__()

#         self.hidden_size = hidden_size

#         self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
#         self.i2o = nn.Linear(input_size + hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden):
#         combined = torch.cat((input, hidden), 1)
#         hidden = self.i2h(combined)
#         output = self.i2o(combined)
#         output = self.softmax(output)
#         return output, hidden

#     def initHidden(self):
#         return torch.zeros(1, self.hidden_size)

# n_hidden = 128
# rnn = RNN(n_letters, n_hidden, n_categories)

In [20]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)
print(output)

tensor([[-2.8803, -2.8161, -2.8097, -2.9276, -2.8146, -2.9399, -2.9498, -2.9254,
         -2.8016, -2.9163, -2.8655, -2.9133, -2.9539, -2.8627, -2.9376, -2.8708,
         -2.9062, -2.9600]], grad_fn=<LogSoftmaxBackward>)


In [25]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.8803, -2.8161, -2.8097, -2.9276, -2.8146, -2.9399, -2.9498, -2.9254,
         -2.8016, -2.9163, -2.8655, -2.9133, -2.9539, -2.8627, -2.9376, -2.8708,
         -2.9062, -2.9600]], grad_fn=<LogSoftmaxBackward>)


# Training

## Preparing for Training

In [26]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Irish', 8)


In [135]:
import random
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Dutch / line = Prinsen
category = Portuguese / line = Pinheiro
category = Japanese / line = Deguchi
category = Japanese / line = Kawate
category = French / line = Romilly
category = Italian / line = Santini
category = German / line = Ott
category = French / line = St pierre
category = Spanish / line = Puerta
category = Chinese / line = Loong


## Training the Network

In [43]:
criterion = nn.NLLLoss()

In [204]:
learning_rate = 0.005  # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    
    rnn.zero_grad()
    
    output = torch.tensor([0])
    for i in range(line_tensor.size()[0]):
#         output, hidden = rnn(line_tensor[i], hidden)
        print('line_tensor[i]:', line_tensor[i])
        print('hidden:', hidden)
        output, hidden = rnn(line_tensor[i], hidden)
#         print(output)
        for p in rnn.parameters():
            print(p, '\n', p.data, '\n', 'p.grad', p.grad)
    
    loss = criterion(output, category_tensor)
#     print(line_tensor)
#     print(output)
#     print(category_tensor)
#     print(loss.item())
    loss.backward()
    
    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        print(p, '\n', p.data, '\n', 'p.grad', p.grad)
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return output, loss.item()

category, line, category_tensor, line_tensor = randomTrainingExample()
output, loss = train(category_tensor, line_tensor)

line_tensor[i]: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
hidden: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
Parameter containing:
tensor([[ 0.0310,  0.0040, -0.0287,  ...,  0.0083, -0.0709,  0.0524],
        [-0.0487, -0.0029,  0.0215,  ..., -0.0342,

In [207]:
category, line, category_tensor, line_tensor = randomTrainingExample()
# output, loss = train(category_tensor, line_tensor)
line_tensor.size()
line_tensor

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0

In [201]:
category, line, category_tensor, line_tensor = randomTrainingExample()
output, loss = train(category_tensor, line_tensor)
# print(loss)
# category
# line_tensor
# print(category_tensor)
# print(line_tensor)

Parameter containing:
tensor([[ 0.0310,  0.0040, -0.0287,  ...,  0.0083, -0.0709,  0.0524],
        [-0.0486, -0.0029,  0.0215,  ..., -0.0342, -0.0499,  0.0516],
        [ 0.0554,  0.0391,  0.0420,  ..., -0.0677, -0.0369, -0.0076],
        ...,
        [ 0.0317,  0.0150, -0.0548,  ...,  0.0349, -0.0205, -0.0505],
        [-0.0064,  0.0693,  0.0316,  ..., -0.0617, -0.0394,  0.0590],
        [ 0.0260, -0.0350, -0.0186,  ...,  0.0293,  0.0279,  0.0666]],
       requires_grad=True) 
 tensor([[ 0.0310,  0.0040, -0.0287,  ...,  0.0083, -0.0709,  0.0524],
        [-0.0486, -0.0029,  0.0215,  ..., -0.0342, -0.0499,  0.0516],
        [ 0.0554,  0.0391,  0.0420,  ..., -0.0677, -0.0369, -0.0076],
        ...,
        [ 0.0317,  0.0150, -0.0548,  ...,  0.0349, -0.0205, -0.0505],
        [-0.0064,  0.0693,  0.0316,  ..., -0.0617, -0.0394,  0.0590],
        [ 0.0260, -0.0350, -0.0186,  ...,  0.0293,  0.0279,  0.0666]]) 
 p.grad tensor([[-1.8914e-03,  0.0000e+00,  0.0000e+00,  ..., -5.8281e-04,
     

In [76]:
import time
import math

n_iters = 100000
# n_iters = 10000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' %(m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    
    # print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, 
                                                timeSince(start), loss, line, guess, correct))
        
    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 8s) 2.0207 Baik / Arabic ✗ (Korean)
10000 10% (0m 17s) 1.2351 Trang / Chinese ✗ (Vietnamese)
15000 15% (0m 25s) 1.5131 Michaud / French ✓
20000 20% (0m 34s) 3.0535 Jordan / Irish ✗ (Polish)
25000 25% (0m 42s) 1.0797 Murakami / Arabic ✗ (Japanese)
30000 30% (0m 51s) 2.2508 Arian / Irish ✗ (Arabic)
35000 35% (1m 0s) 2.1622 Trujillo / Italian ✗ (Spanish)
40000 40% (1m 8s) 0.5339 Xie / Chinese ✓
45000 45% (1m 16s) 1.0396 Bernard / French ✓
50000 50% (1m 24s) 0.6939 An / Vietnamese ✓
55000 55% (1m 33s) 0.1506 Yamamura / Japanese ✓
60000 60% (1m 41s) 0.6029 Albuquerque / Portuguese ✓
65000 65% (1m 50s) 2.4214 Janda / Czech ✗ (Polish)
70000 70% (1m 59s) 0.2806 Gui / Chinese ✓
75000 75% (2m 8s) 0.0259 Astrakhantsev / Russian ✓
80000 80% (2m 16s) 0.8768 Svejkovsky / Czech ✓
85000 85% (2m 24s) 1.2164 Fuchs / Portuguese ✗ (German)
90000 90% (2m 33s) 3.2359 Smit / French ✗ (Dutch)
95000 95% (2m 41s) 0.9262 Jordison / English ✓
100000 100% (2m 49s) 0.8570 Wehner / German ✓


# My data for two-phase flow

## Training

In [481]:
# Read data file
import pandas as pd
df_features_label = pd.read_csv('df_train_set_features_label_time_series.csv', index_col=0)
pd.set_option('display.max_columns', 12)
df_features_label

0        1        2        3        4        5  ...     1495  \
64   1.02711  1.02736  1.02686  1.03026  1.02794  1.02596  ...  0.54360   
26   0.16582  0.15830  0.16184  0.16381  0.16471  0.16525  ...  0.19562   
22   0.18607  0.19869  0.19684  0.18523  0.18014  0.18466  ...  0.18651   
31   0.17047  0.18804  0.18990  0.19283  0.19455  0.18451  ...  0.18701   
47   0.41909  0.42145  0.42786  0.44190  0.45160  0.45377  ...  0.26470   
..       ...      ...      ...      ...      ...      ...  ...      ...   
71   0.44239  0.44237  0.44241  0.44240  0.44237  0.44241  ...  0.50666   
14   0.21375  0.21745  0.22284  0.22201  0.22001  0.21662  ...  0.19529   
92   1.03805  1.04144  1.03779  1.03940  1.03712  1.03881  ...  1.02976   
51   0.23968  0.23916  0.23897  0.23970  0.23995  0.23952  ...  0.49906   
102  1.03426  1.03661  1.03560  1.03334  1.03460  1.03451  ...  1.03771   

        1496     1497     1498     1499  label  
64   0.54218  0.54270  0.54080  0.53965      4  
26   0.19314  0.19567  0.19845  0.20544      2  
22   0.18654  0.18681  0.19088  0.19802      2  
31   0.18644  0.18176  0.18024  0.17893      2  
47   0.26405  0.26547  0.26552  0.26478      3  
..       ...      ...      ...      ...    ...  
71   0.50658  0.50497  0.50340  0.50495      4  
14   0.19605  0.19698  0.19731  0.19738      1  
92   1.03560  1.03695  1.03351  1.03535      6  
51   0.49224  0.48130  0.48296  0.48725      3  
102  1.03966  1.03898  1.03393  1.03510      6  

[86 rows x 1501 columns]

In [82]:
df_features_label.values

array([[1.02711, 1.02736, 1.02686, ..., 0.5408 , 0.53965, 4.     ],
       [0.16582, 0.1583 , 0.16184, ..., 0.19845, 0.20544, 2.     ],
       [0.18607, 0.19869, 0.19684, ..., 0.19088, 0.19802, 2.     ],
       ...,
       [1.03805, 1.04144, 1.03779, ..., 1.03351, 1.03535, 6.     ],
       [0.23968, 0.23916, 0.23897, ..., 0.48296, 0.48725, 3.     ],
       [1.03426, 1.03661, 1.0356 , ..., 1.03393, 1.0351 , 6.     ]])

In [260]:
# Convert to Tensor
data = torch.tensor(df_features_label.values, dtype=torch.double)
data
data.size()

torch.Size([86, 1501])

In [ ]:
n_iters = 20

In [101]:
data[0][-1:]

tensor([4.], dtype=torch.float64)

In [102]:
data[0][:-1]

tensor([1.0271, 1.0274, 1.0269,  ..., 0.5427, 0.5408, 0.5396],
       dtype=torch.float64)

In [475]:
# Create RNN neural network

import torch.nn as nn

class MY_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MY_RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
#         self.i3h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
#         hidden = self.i3h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_voltage = 1
n_hidden = 128
n_categories = 6
my_rnn = MY_RNN(n_voltage, n_hidden, n_categories)

In [476]:
a = (1, 2, 3)
b = torch.tensor(a)
type(b)

# learning_rate = 0.005  # If you set this too high, it might explode. If too low, it might not learn

# Define training function

criterion = nn.NLLLoss()

learning_rate = 0.001 # If you set this too high, it might explode. If too low, it might not learn

def my_train(category_tensor, line_tensor):
    hidden = my_rnn.initHidden()

    my_rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = my_rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in my_rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()


# category_tensor, line_tensor = data[0][-1:].to(dtype=torch.long), data[0][:-1].view(-1, 1, 1).to(dtype=torch.float32)
# output, loss = my_train(category_tensor, line_tensor)

In [477]:
category_tensor, line_tensor = data[0][-1:].to(dtype=torch.long), data[0][:-1].to(dtype=torch.float32)
line_tensor.size()[0]

# n_hidden = 128
# hidden =torch.zeros(1, n_hidden)
# input = torch.tensor([[0.5]])
# output, nex_hidden = my_rnn(input, hidden)
# # torch.cat((input, hidden), 1)

category_tensor, line_tensor = data[0][-1:].to(dtype=torch.long), data[0][:-1].view(-1, 1, 1).to(dtype=torch.float32)
output, loss = my_train(category_tensor, line_tensor)
output
loss

1.8554799556732178

In [478]:
# 数字1-6分别代表层状流、环状流、波状流、塞状流、弹状流和泡状流
# Train the model
my_all_categories = ['stratified', 'annular', 'wave', 'plug', 'slug', 'bubble']

def my_categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return my_all_categories[category_i], category_i

def my_randomTrainingExample():
    num = random.randint(0, 85) # 86 training samples
    category_tensor = data[num][-1:].to(dtype=torch.long) - 1
    category= category_tensor.item()
    line_tensor= data[num][:-1].view(-1, 1, 1).to(dtype=torch.float32)
    line = line_tensor
    return category, line, category_tensor, line_tensor

my_categoryFromOutput(output)
# my_randomTrainingExample()

import time
import math

# n_iters = 86
n_iters = 1000
print_every = 10
plot_every = 10

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    """record time from start to now in mm ss."""
    now = time.time()
    s = now -since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    i = iter % 86
    category = int(df_features_label.values[i][-1] - 1)
    category_tensor= data[i][-1:].to(dtype=torch.long) - 1
    line_tensor = data[i][:-1].view(-1, 1, 1).to(dtype=torch.float32)
#     category, line, category_tensor, line_tensor = my_randomTrainingExample()
    output, loss = my_train(category_tensor, line_tensor)
    current_loss =+ loss

    # Print iter number, loss, regime and guess
    if iter % print_every == 0:
        guess, guess_i = my_categoryFromOutput(output)
        correct = '✓' if guess_i == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f / %s %s' % (iter, iter / n_iters * 100, 
                timeSince(start), loss, guess_i, correct))

#     # Add current loss avg to list of losses
#     if iter % plot_every == 0:
#         all_losses.append(current_loss / plot_every)
#         current_loss = 0

10 1% (0m 3s) 1.7926 / 2 ✗ (0)
20 2% (0m 7s) 1.8077 / 2 ✗ (1)
30 3% (0m 10s) 1.8050 / 2 ✗ (1)
40 4% (0m 14s) 1.7080 / 2 ✓
50 5% (0m 17s) 1.7150 / 2 ✓
60 6% (0m 20s) 1.8985 / 2 ✗ (4)
70 7% (0m 24s) 1.6955 / 2 ✓
80 8% (0m 27s) 1.7895 / 2 ✗ (3)
90 9% (0m 31s) 1.7191 / 2 ✓
100 10% (0m 34s) 1.8072 / 2 ✗ (1)
110 11% (0m 38s) 1.8285 / 2 ✗ (3)
120 12% (0m 41s) 1.7831 / 2 ✗ (3)
130 13% (0m 45s) 1.8277 / 2 ✗ (3)
140 14% (0m 48s) 1.8961 / 2 ✗ (4)
150 15% (0m 52s) 1.7755 / 2 ✗ (3)
160 16% (0m 55s) 1.8906 / 2 ✗ (4)
170 17% (0m 59s) 1.7084 / 2 ✓
180 18% (1m 2s) 1.8569 / 2 ✗ (4)
190 19% (1m 6s) 1.7537 / 2 ✗ (5)
200 20% (1m 9s) 1.8618 / 2 ✗ (4)
210 21% (1m 12s) 1.8010 / 2 ✗ (0)
220 22% (1m 16s) 1.7223 / 2 ✓
230 23% (1m 19s) 1.7452 / 2 ✗ (5)
240 24% (1m 23s) 1.7376 / 2 ✗ (5)
250 25% (1m 26s) 1.8855 / 2 ✗ (4)
260 26% (1m 30s) 1.8040 / 2 ✗ (1)
270 27% (1m 33s) 1.7798 / 2 ✗ (3)
280 28% (1m 37s) 1.8783 / 2 ✗ (4)
290 28% (1m 40s) 1.7292 / 2 ✗ (5)
300 30% (1m 44s) 1.8735 / 2 ✗ (4)
310 31% (1m 47s) 1.8193 / 2

In [427]:
4 % 3

1

## Testing

In [258]:
import pandas as pd
df_test_features_label = pd.read_csv('df_test_set_feature_label_time_series.csv', index_col=0)
# df_test_features_label.head()

test_data = torch.tensor(df_test_features_label.values, dtype=torch.double)
test_data.size()

# category_tensor, line_tensor = test_data[0][-1:].to(dtype=torch.long), test_data[0][:-1].to(dtype=torch.float32)
# line_tensor.size()[0]

torch.Size([22, 1501])

In [257]:
def my_evaluate(line_tensor):
    hidden = my_rnn.initHidden()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = my_rnn(line_tensor[i], hidden)
    
    return output

def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = my_evaluate(input_line)
        
        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []
        
        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, my_all_categories[category_index]))
            predictions.append([value, my_all_categories[category_index]])

category_tensor, line_tensor = test_data[0][-1:].to(dtype=torch.long), test_data[0][:-1].view(-1, 1, 1).to(dtype=torch.float32)
predict(line_tensor)


> tensor([[[0.5773]],

        [[0.5764]],

        [[0.5739]],

        ...,

        [[0.5863]],

        [[0.5889]],

        [[0.5890]]])
(-1.72) annular
(-1.75) stratified
(-1.78) bubble


In [487]:
n_iters = 22
for iter in range(1, n_iters + 1):
    category = int(df_test_features_label.values[iter - 1][-1:])
    line_tensor = data[iter - 1][:-1].view(-1, 1, 1).to(dtype=torch.float32)
#     output, loss = my_train(category_tensor, line_tensor)
#     current_loss =+ loss
    output = my_evaluate(line_tensor)
#     print(output)
    
    # Print iter number, loss, regime and guess
#     if iter % print_every == 0:
    guess, guess_i = my_categoryFromOutput(output)
    category_name = my_all_categories[category - 1]
    correct = '✔' if guess == category_name else '✖ (%s)' % category_name
    print('%d %d%% (%s) / %s %s' % (iter, iter / n_iters * 100, 
            timeSince(start), guess, correct))
        
#     # Add current loss avg to list of losses
#     if iter % plot_every == 0:
#         all_losses.append(current_loss / plot_every)
#         current_loss = 0

1 4% (469m 41s) / bubble ✖ (slug)
2 9% (469m 41s) / wave ✖ (stratified)
3 13% (469m 41s) / wave ✖ (stratified)
4 18% (469m 42s) / wave ✖ (slug)
5 22% (469m 42s) / wave ✖ (plug)
6 27% (469m 42s) / bubble ✖ (plug)
7 31% (469m 42s) / wave ✖ (annular)
8 36% (469m 42s) / bubble ✖ (wave)
9 40% (469m 43s) / bubble ✔
10 45% (469m 43s) / bubble ✖ (stratified)
11 50% (469m 43s) / wave ✖ (slug)
12 54% (469m 43s) / wave ✖ (slug)
13 59% (469m 43s) / bubble ✖ (stratified)
14 63% (469m 44s) / bubble ✖ (slug)
15 68% (469m 44s) / wave ✖ (annular)
16 72% (469m 44s) / bubble ✖ (plug)
17 77% (469m 44s) / wave ✖ (plug)
18 81% (469m 44s) / wave ✖ (slug)
19 86% (469m 45s) / bubble ✔
20 90% (469m 45s) / bubble ✖ (wave)
21 95% (469m 45s) / wave ✖ (stratified)
22 100% (469m 45s) / wave ✔
